In [1]:
import QuantLib as ql
from matplotlib import pyplot as plt
import numpy as np
import math

%matplotlib inline

# Build the Process

In [3]:
todaysDate = ql.Date(15, ql.May, 2023)
ql.Settings.instance().evaluationDate = todaysDate

## Option construction
exercise = ql.AmericanExercise(todaysDate, ql.Date(17, ql.May, 2024))
payoff = ql.PlainVanillaPayoff(ql.Option.Put, 40.0)
option = ql.VanillaOption(payoff, exercise)

# Market data
underlying = ql.SimpleQuote(36.0)
dividendYield = ql.FlatForward(todaysDate, 0.00, ql.Actual365Fixed())
volatility = ql.BlackConstantVol(todaysDate, ql.TARGET(), 0.20, ql.Actual365Fixed())
riskFreeRate = ql.FlatForward(todaysDate, 0.06, ql.Actual365Fixed())

# black scholes process contains the term structures
process = ql.BlackScholesMertonProcess(
    ql.QuoteHandle(underlying),
    ql.YieldTermStructureHandle(dividendYield),
    ql.YieldTermStructureHandle(riskFreeRate),
    ql.BlackVolTermStructureHandle(volatility),
)

## Build the QD Plus Engine

In [ ]:
qdplus_engine =  ql.QdPlusAmericanEngine(
    process,
    interpolationPoints = 8,
    solverType = ql.QdPlusAmericanEngine.Halley,
    eps = 1e-6,
    maxIter = None
)

In [ ]:
results = []

#### Analytic approximations
option.setPricingEngine(ql.BaroneAdesiWhaleyApproximationEngine(process))
results.append(("Barone-Adesi-Whaley", option.NPV()))

option.setPricingEngine(ql.BjerksundStenslandApproximationEngine(process))
results.append(("Bjerksund-Stensland", option.NPV()))

In [ ]:
#### Finite-difference method
timeSteps = 801
gridPoints = 800
dampingSteps = 0
schemeDesc = ql.DouglasScheme()
fd_engine = ql.FdBlackScholesVanillaEngine(process, timeSteps, gridPoints, dampingSteps, schemeDesc)

option.setPricingEngine(fd_engine)
results.append(("finite differences", option.NPV()))

# Linear Regression

In [ ]:
ql.LsmBasisSystem.Monomial 
ql.LsmBasisSystem.Laguerre
ql.LsmBasisSystem.Hermite
ql.LsmBasisSystem.Hyperbolic
ql.LsmBasisSystem.Chebyshev2nd

coeff_[i-1] = GeneralLinearLeastSquares(x, y, v_).coefficients()


In [3]:
# Longstaff Schwartz
mc_engine = ql.MCAmericanEngine(
    process,
    "pseudorandom",
    timeSteps=timeSteps,
    timeStepsPerYear=None,
    antitheticVariate=False,
    controlVariate=False,
    requiredSamples=None,
    requiredTolerance=0.02,
    maxSamples=None,
    seed=42,
    polynomOrder=3,
    polynomType=ql.LsmBasisSystem.Monomial,
    antitheticVariateCalibration=None,
    seedCalibration=None
)

option.setPricingEngine(mc_engine)

results.append(("LS American", option.NPV()))

[('Barone-Adesi-Whaley', 4.463536264085715),
 ('Bjerksund-Stensland', 4.4569588667986935),
 ('finite differences', 4.489992153451754),
 ('QD+', 4.501016922307475),
 ('QD+ fixed point', 4.490552145592267)]

In [ ]:
# #### Li, M. QD+ American engine
qdplus_engine =  ql.QdPlusAmericanEngine(
    process,
    interpolationPoints = 8,
    solverType = ql.QdPlusAmericanEngine.Halley,
    eps = 1e-6,
    maxIter = None
)
option.setPricingEngine(qdplus_engine)
results.append(("QD+", option.NPV()))

In [ ]:
# #### Leif Andersen, Mark Lake and Dimitri Offengenden high performance American engine

# Gauss-Legendre (l,m,n)-p Scheme
#        \param l  order of Gauss-Legendre integration within every fixed point iteration step
#        \param m  fixed point iteration steps, first step is a partial Jacobi-Newton,
#                  the rest are naive Richardson fixed point iterations
#        \param n  number of Chebyshev nodes to interpolate the exercise boundary
#        \param p  order of Gauss-Legendre integration in final conversion of the
#                  exercise boundary into option prices

ql.QdFpIterationScheme, ql.QdFpLegendreScheme, ql.QdFpLegendreTanhSinhScheme, ql.QdFpTanhSinhIterationScheme
# ql.QdFpAmericanEngine.fastScheme(), ql.QdFpAmericanEngine.highPrecisionScheme()

qdf_engine = ql.QdFpAmericanEngine(process, ql.QdFpAmericanEngine.accurateScheme()) 

option.setPricingEngine(qdf_engine)

results.append(("QD+ fixed point", option.NPV()))

## Build Iteration Scheme

In [ ]:
def calculatePut(S, K, r, q, vol, T, iteration_scheme, process, QdPlusAmericanEngine) -> float:
   
    if (r < 0.0) and (q < r):
        raise Exception, "double-boundary case q<r<0 for a put option is given"

    xmax = QdPlusAmericanEngine.xMax(K, r, q)
    n = iteration_scheme.getNumberOfChebyshevInterpolationNodes()

    interp: ql.ChebyshevInterpolation = ql.QdPlusAmericanEngine(
        process, 
        n+1, 
        ql.QdPlusAmericanEngine.Halley, 
        1e-8
    ).getPutExerciseBoundary(S, K, r, q, vol, T)

    z: ql.Array = interp.nodes()
    x: ql.Array = 0.5*np.sqrt(T)*(1.0+z)

    # Boundary
    B = lambda xmax, T, interp, tau: xmax * np.exp(-np.sqrt(np.max(0, interp(2*np.sqrt(np.abs(tau)/T)-1, True))))

    h = lambda fv, xmax : np.squared(np.log(fv/xmax))
        
    # eqn: ql.DqFpEquation = (fpEquation_ == FP_A || (fpEquation_ == Auto && np.abs(r-q) < 0.001)) ? DqFpEquation_A(K, r, q, vol, B, iterationScheme_->getFixedPointIntegrator()) : DqFpEquation_B(K, r, q, vol, B, iterationScheme_->getFixedPointIntegrator()))

    eqn = ql.DqFpEquation_A(K, r, q, vol, B, iteration_scheme.getFixedPointIntegrator())
    y = ql.Array(x.size())
    y[0] = 0.0

    n_newton = iteration_scheme.getNumberOfJacobiNewtonFixedPointSteps()
    for k in range(n_newton):
        for i in range(1, x.size()):
            tau = np.square(x[i])
            b = B(tau)
            
            N, D, fv = eqn.f(tau, b)
           
            if (tau < QL_EPSILON):
                y[i] = h(fv)
            else:
                Nd, Dd = eqn.NDd(tau, b)
                # Nd = std::get<0>(ndd)
                # Dd = std::get<1>(ndd)
                fd = K * np.exp(-(r-q) * tau) * (Nd/D - Dd*N/(D*D))
                y[i] = h(b - (fv - b)/ (fd-1))
        interp.updateY(y)

    n_fp = iteration_scheme.getNumberOfNaiveFixedPointSteps()
    for k in range(n_fp):
        for i in range(1, x.size()):
            tau = np.squared(x[i])
            _, fv = eqn.f(tau, B(tau))
            y[i] = h(fv)
        interp.updateY(y)
        
    # detail::QdPlusAddOnValue aov(T, S, K, r, q, vol, xmax, interp)
    # addOn = (*iteration_scheme.getExerciseBoundaryToPriceIntegrator())(aov, 0.0, np.sqrt(T))

    europeanValue = ql.BlackCalculator(ql.Option.Put, K, S * np.exp((r-q)*T), vol*np.sqrt(T), np.exp(-r*T)).value()

    return np.max(europeanValue, 0.0) + np.max(0.0, addOn)

